### Aula 4 - Testes Estatísticos Aplicados com dataset NHANES 2015-2016

"""

O script cobre:

1. Carregamento e preparação dos dados NHANES
2. Formulação de hipóteses, alfa e valor-p(p-value)
3. t-tests:
   - t-test Simples (One Sample) 
   - t-test independente(não-pareado)
   - t-test dependente (pareado)
4. Testes de normalidade:
   - Shapiro-Wilk
   - Kolmogorov-Smirnov
   - Anderson-Darling
   - Histogramas e QQ-Plot
5. Teste Qui-Quadrado(X²) de independência
6. Teste de proporções / Survival Function (binomial)
7. ANOVA (uma via)
8. Teorema do Limite Central (CLT) com simulação
9. Lei dos Grandes Números (LLN)
10. Exemplos de amostragem simples e estratificada
11. 
"""

In [0]:
# =====================================
# 0. IMPORTAÇÃO DE BIBLIOTECAS
# =====================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
# Para QQ-plot
import statsmodels.api as sm

# =====================================
# 1. CONFIGURAÇÃO BÁSICA
# =====================================

# Nome do arquivo NHANES (ajuste o caminho se necessário)
NHANES_PATH = "nhanes_2015_2016.csv"

# Nomes de colunas usados (do CSV em português)
COL_ID       = "ID_Participante"
COL_ALCOOL   = "ConsumiuAlcool12Meses"
COL_FUMOU    = "JaFumou100Cigarros"
COL_SEXO     = "Sexo"
COL_IDADE    = "Idade"
COL_IMC      = "IMC"
COL_PESO     = "Peso_kg"
COL_ALTURA   = "Altura_cm"
COL_RACA     = "RacaEtnia"
COL_ESCOLAR  = "Escolaridade"
COL_BP_SYS1  = "PressaoSistolica_1"
COL_BP_DIA1  = "PressaoDiastolica_1"
COL_BP_SYS2  = "PressaoSistolica_2"
COL_BP_DIA2  = "PressaoDiastolica_2"

In [0]:
# =====================================
# 2. CARREGAMENTO E PRÉ-PROCESSAMENTO
# =====================================

df = pd.read_csv(NHANES_PATH)

print("Tamanho original do dataset:", df.shape)
print("\nColunas disponíveis:\n", df.columns)

In [0]:
# Vamos considerar apenas adultos (Idade >= 18) para vários testes
df_adultos = df[df[COL_IDADE] >= 18].copy()

# Remover linhas com NaN nas colunas relevantes
cols_importantes = [
                    COL_IDADE, COL_SEXO, COL_IMC, COL_PESO, COL_ALTURA,
                    COL_FUMOU, COL_RACA, COL_ESCOLAR,
                    COL_BP_SYS1, COL_BP_DIA1, COL_BP_SYS2, COL_BP_DIA2
                    ]

cols_existentes = [c for c in cols_importantes if c in df_adultos.columns]
df_adultos = df_adultos.dropna(subset=cols_existentes)

print("Tamanho após filtro de adultos + remoção de NaNs:", df_adultos.shape)

In [0]:
# ===========================================
# 3. EXEMPLO DE HIPÓTESE, ALFA E VALOR-p
#    t-test Simples - (One Sample) sobre o IMC
# ===========================================

"""
Pergunta:
A média do IMC dos adultos da amostra NHANES é igual a 25?

H0 (hipótese nula): μ_IMC = 25
HA (hipótese alternativa): μ_IMC ≠ 25

Usamos t-test Simples (One way Sample).
"""

alpha = 0.05  # nível de significância (5%)

imc = df_adultos[COL_IMC]

t_stat, p_value = stats.ttest_1samp(imc, popmean=25)

print("\n=== t-test Smples (One Way Sample) para IMC = 25 ===")
print(f"t estatístico: {t_stat:.3f}")
print(f"p-valor (bicaudal): {p_value:.5f}")
print(f"Nível de significância (alfa): {alpha}")

if p_value < alpha:
    print("Conclusão HA: rejeitamos H0 -> a média de IMC é diferente de 25.")
else:
    print("Conclusão H0: não rejeitamos H0 (aceitamos H0) -> não há evidência(estatística) suficiente de que a média de IMC seja diferente de 25.")

# Distribuição do IMC (visual)
plt.figure()
plt.hist(imc, bins=30, edgecolor="black", density=True)
plt.title("Distribuição do IMC (Adultos)")
plt.xlabel("IMC")
plt.ylabel("Densidade");

In [0]:
# ===========================================
# 4. t-test INDEPENDENTE (IMC por sexo)
# ===========================================

"""
Pergunta:
Homens e mulheres têm IMC médio diferente?

H0: μ_IMC_homens = μ_IMC_mulheres
HA: μ_IMC_homens ≠ μ_IMC_mulheres
"""

# Supondo codificação: 1 = Masculino, 2 = Feminino (verificar no dataset)
imc_homens   = df_adultos[df_adultos[COL_SEXO] == 1][COL_IMC]
imc_mulheres = df_adultos[df_adultos[COL_SEXO] == 2][COL_IMC]

t_stat_ind, p_value_ind = stats.ttest_ind(imc_homens, imc_mulheres, equal_var=False)

print("\n=== t-test INDEPENDENTE: IMC por sexo (Homens vs Mulheres) ===")
print(f"N homens: {len(imc_homens)}, N mulheres: {len(imc_mulheres)}")
print(f"IMC médio homens:   {imc_homens.mean():.2f}")
print(f"IMC médio mulheres: {imc_mulheres.mean():.2f}")
print(f"t estatístico: {t_stat_ind:.3f}")
print(f"p-valor: {p_value_ind:.5f}")

#Codificando o teste de hipoteses propriamente dito - H0 vs HA
if p_value_ind < alpha:
    print("Conclusão HA: rejeitamos H0 -> há diferença significativa de IMC entre homens e mulheres.")
else:
    print("Conclusão H0: não rejeitamos H0 -> não há evidência suficiente de diferença de IMC entre homens e mulheres.")

# Boxplot IMC por sexo
plt.figure()
df_adultos.boxplot(column=COL_IMC, by=COL_SEXO)
plt.title("IMC por Sexo (1=Homens, 2=Mulheres)")
plt.suptitle("")
plt.xlabel("Sexo")
plt.ylabel("IMC");

In [0]:
# ===========================================
# 5. t-test DEPENDENTE (pareado) - Pressão
# ===========================================

"""
Exemplo tipo "antes e depois":
Suponha que PressaoSistolica_1 e PressaoSistolica_2 sejam duas medidas
da mesma pessoa em momentos diferentes (ex.: antes e depois de uma intervenção).

H0: a média da diferença = 0 (não houve mudança real)
HA: a média da diferença ≠ 0
"""

bps1 = df_adultos[COL_BP_SYS1]
bps2 = df_adultos[COL_BP_SYS2]

t_stat_paired, p_value_paired = stats.ttest_rel(bps1, bps2)

print("\n=== t-test DEPENDENTE: Pressão Sistólica 1 vs 2 ===")
print(f"t estatístico: {t_stat_paired:.3f}")
print(f"p-valor: {p_value_paired:.5f}")

#Codificando o teste de hipoteses propriamente dito - H0 vs HA
if p_value_paired < alpha:
    print("Conclusão HA: rejeitamos H0 -> há diferença significativa entre as duas medições de pressão.")
else:
    print("Conclusão H0: não rejeitamos H0 -> sem evidência forte de diferença entre as medições.")

In [0]:
# ===========================================
# 6. TESTES DE NORMALIDADE PARA IMC
# ===========================================

"""
H0: os dados seguem distribuição Normal
HA: os dados NÃO seguem distribuição Normal
"""

print("\n=== Testes de Normalidade para IMC ===")

# 6.1 Shapiro-Wilk (bom para amostras pequenas a moderadas; aqui é só demonstrativo)
# Para não sobrecarregar, podemos pegar uma amostra aleatória de 500 observações
imc_amostra = imc.sample(n=min(500, len(imc)), random_state=42)

shapiro_stat, shapiro_p = stats.shapiro(imc_amostra)
print(f"Shapiro-Wilk: estat = {shapiro_stat:.3f}, p-valor = {shapiro_p:.5f}")

# 6.2 Kolmogorov-Smirnov test com parâmetros ajustados pela própria amostra
mu, sigma = imc.mean(), imc.std(ddof=1)
ks_stat, ks_p = stats.kstest(imc, 'norm', args=(mu, sigma))
print(f"Kolmogorov-Smirnov: estat = {ks_stat:.3f}, p-valor = {ks_p:.5f}")

# 6.3 Anderson-Darling
anderson_result = stats.anderson(imc, dist='norm')
print("Anderson-Darling: estatística =", anderson_result.statistic)
for i, (cv, sig) in enumerate( zip(anderson_result.critical_values, anderson_result.significance_level) ):
    print(f"  Nível de significância {sig}% -> valor crítico {cv:.3f}")

# 6.4 Histograma
plt.figure()
plt.hist(imc, bins=30, edgecolor="black")
plt.title("Histograma do IMC")
plt.xlabel("IMC")
plt.ylabel("Frequência")

#QQ-Plot
sm.qqplot(imc, line='s')
plt.title("QQ-Plot do IMC");

In [0]:

# ===========================================
# 7. TESTE QUI-QUADRADO DE INDEPENDÊNCIA
#    Ex.: Sexo x Já Fumou 100 cigarros
# ===========================================

"""
H0: Sexo e "Já fumou 100 cigarros" são independentes
HA: Há associação entre Sexo e "Já fumou 100 cigarros"
"""

# Criar tabela de contingência
contingencia = pd.crosstab(df_adultos[COL_SEXO], df_adultos[COL_FUMOU])
chi2_stat, chi2_p, chi2_dof, chi2_exp = stats.chi2_contingency(contingencia)

print("\n=== Teste Qui-Quadrado de Independência: Sexo x JaFumou100Cigarros ===")
print("Tabela de contingência:\n", contingencia)
print(f"\nQui-Quadrado: {chi2_stat:.3f}")
print(f"Graus de liberdade: {chi2_dof}")
print(f"p-valor: {chi2_p:.5f}")

if chi2_p < alpha:
    print("Conclusão H0: rejeitamos H0 -> há associação entre Sexo e hábito de fumar 100 cigarros na vida.")
else:
    print("Conclusão HA: não rejeitamos H0 -> não há evidência forte de associação.")

In [0]:
# ===========================================
# 8. TESTE DE PROPORÇÕES / SURVIVAL FUNCTION
# ===========================================

"""
Exemplo: qual a probabilidade de observar pelo menos k fumantes em uma amostra de n,
assumindo que a proporção verdadeira seja p0?
Usamos a distribuição Binomial e a Survival Function (sf) = P(X >= k).
"""

# Proporção observada de fumantes (JaFumou100Cigarros == 1)
fumantes = (df_adultos[COL_FUMOU] == 1).sum()
n_total  = len(df_adultos)
p_observado = fumantes / n_total

print("\n=== Proporção de fumantes (JaFumou100Cigarros == 1) ===")
print(f"Fumantes: {fumantes}, Total: {n_total}, p_observado = {p_observado:.3f}")

# Suponha que a proporção "real" seja p0 = 0.2 (20%)
p0 = 0.2
k = fumantes     # número de "sucessos" observados

# Survival function: P(X >= k) para X ~ Binomial(n_total, p0)
prob_sf = stats.binom.sf(k-1, n_total, p0)  # sf(k-1) = P(X >= k)

print(f"Supondo p0 = {p0:.2f}, a probabilidade de observar {k} ou mais fumantes em {n_total} pessoas é {prob_sf:.6f}")

In [0]:
# ===========================================
# 9. ANOVA (One Way Sample) DE UMA VIA
#    Ex.: IMC em função da Raça/Etnia
# ===========================================

"""
H0: todas as médias de IMC nos grupos de Raça/Etnia são iguais
HA: pelo menos um grupo tem média de IMC diferente
"""

# Selecionar alguns grupos com tamanho razoável
grupos_imc = []
labels_grupos = []

for raca, subdf in df_adultos.groupby(COL_RACA):
    if len(subdf) > 50:  # mínimo de 50 indivíduos por grupo, só para robustez
        grupos_imc.append(subdf[COL_IMC].values)
        labels_grupos.append(raca)

if len(grupos_imc) >= 2:
    f_stat, p_anova = stats.f_oneway(*grupos_imc)
    print("\n=== ANOVA de uma via: IMC por Raça/Etnia ===")
    print("Grupos considerados:", labels_grupos)
    print(f"F estatístico: {f_stat:.3f}")
    print(f"p-valor: {p_anova:.5f}")
    if p_anova < alpha:
        print("Conclusão: rejeitamos H0 -> pelo menos um grupo de Raça/Etnia tem IMC médio diferente.")
    else:
        print("Conclusão: não rejeitamos H0 -> sem evidência forte de diferenças de médias entre os grupos.")
else:
    print("\nANOVA não realizada: poucos grupos com tamanho amostral suficiente.")

# Boxplot IMC por Raça/Etnia (apenas visual)
plt.figure(figsize=(10, 5))
df_adultos.boxplot(column=COL_IMC, by=COL_RACA, rot=45)
plt.title("IMC por Raça/Etnia")
plt.suptitle("")
plt.xlabel("Raça/Etnia")
plt.ylabel("IMC")
plt.tight_layout();

In [0]:
# ===========================================
# 10. TEOREMA DO LIMITE CENTRAL (CLT)
# ===========================================

"""
Simulação com o IMC:
- Tiramos várias amostras aleatórias de tamanho n
- Calculamos a média de cada amostra
- Mostramos a distribuição das médias amostrais
- A ideia é que a distribuição dessas médias se aproxime de uma Normal
"""

n = 50          # tamanho de cada amostra
num_amostras = 1000

medias_amostrais = []
for i in range(num_amostras):
    amostra = imc.sample(n=n, replace=True)
    medias_amostrais.append(amostra.mean())

medias_amostrais = np.array(medias_amostrais)

plt.figure()
plt.hist(medias_amostrais, bins=30, edgecolor="black", density=True)
plt.title(f"CLT - Distribuição das médias amostrais de IMC (n={n})")
plt.xlabel("Média do IMC")
plt.ylabel("Densidade")


# QQ-plot das médias amostrais
sm.qqplot(medias_amostrais, line='s')
plt.title("QQ-Plot das médias amostrais de IMC (CLT)")

In [0]:
# ===========================================
# 11. LEI DOS GRANDES NÚMEROS (LLN)
# ===========================================

"""
Exemplo com o IMC:
- Embaralhamos a ordem dos indivíduos
- Calculamos a média acumulada conforme vamos adicionando observações
- A média acumulada tende ao valor real da média populacional (amostra total)
"""

imc_embaralhado = imc.sample(frac=1.0, random_state=42).values
medias_acumuladas = np.cumsum(imc_embaralhado) / (np.arange(len(imc_embaralhado)) + 1)
media_global = imc.mean()

plt.figure()
plt.plot(medias_acumuladas, label="Média acumulada")
plt.axhline(media_global, linestyle="--", label=f"Média global = {media_global:.2f}")
plt.title("Lei dos Grandes Números - IMC")
plt.xlabel("Número de observações")
plt.ylabel("Média")
plt.legend();

In [0]:
# ===========================================
# 12. AMOSTRAGEM SIMPLES E ESTRATIFICADA
# ===========================================

# 12.1 Amostragem Aleatória Simples
tamanho_amostra = 200
amostra_simples = df_adultos.sample(n=tamanho_amostra, random_state=42)

print("\n=== Amostragem Aleatória Simples ===")
print("Tamanho da amostra:", len(amostra_simples))
print("IMC médio na amostra simples:", amostra_simples[COL_IMC].mean())

# 12.2 Amostragem Estratificada por Sexo (proporcional)
print("\n=== Amostragem Estratificada por Sexo (proporcional) ===")

# Proporção de cada sexo na base
proporcoes = df_adultos[COL_SEXO].value_counts(normalize=True)
print("Proporções na população (dataset adultos):")
print(proporcoes)

# Queremos também ~200 observações, mas respeitando proporções
amostras_estratos = []
for sexo, prop in proporcoes.items():
    n_estrato = int(round(tamanho_amostra * prop))
    subdf = df_adultos[df_adultos[COL_SEXO] == sexo]
    amostra_estrato = subdf.sample(n=min(n_estrato, len(subdf)), random_state=42)
    amostras_estratos.append(amostra_estrato)

amostra_estratificada = pd.concat(amostras_estratos)
print("Tamanho da amostra estratificada:", len(amostra_estratificada))
print("IMC médio na amostra estratificada:", amostra_estratificada[COL_IMC].mean())
print("Proporções de sexo na amostra estratificada:")
print(amostra_estratificada[COL_SEXO].value_counts(normalize=True))

print("\nScript concluído.")

### FIM